In [56]:
import pandas as pd
import numpy as np
import os

In [57]:
root_dir = "D:/vhproj/Ton_IoT"
datadir = "data/IoT ID20/IoT Network Intrusion Dataset.csv"

path = os.path.join(root_dir, datadir)

print(path)
# Read data and cleaning
df = pd.read_csv(path)
df.replace('-', np.nan, inplace=True)
for c in df.select_dtypes(include=['object']).columns:
    df[c] = df[c].astype(str).str.strip()
print(df.columns)
print(df.info())
print(df.head(5))

# Kiểm tra số lượng loại nhãn trong Label
print("Các giá trị nhãn Label và số lượng:")
print(df['Label'].value_counts())

D:/vhproj/Ton_IoT\data/IoT ID20/IoT Network Intrusion Dataset.csv
Index(['Flow_ID', 'Src_IP', 'Src_Port', 'Dst_IP', 'Dst_Port', 'Protocol',
       'Timestamp', 'Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts',
       'TotLen_Fwd_Pkts', 'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Max',
       'Fwd_Pkt_Len_Min', 'Fwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std',
       'Bwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Std', 'Flow_Byts/s', 'Flow_Pkts/s', 'Flow_IAT_Mean',
       'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Tot',
       'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Fwd_IAT_Min',
       'Bwd_IAT_Tot', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Bwd_IAT_Max',
       'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags', 'Fwd_URG_Flags',
       'Bwd_URG_Flags', 'Fwd_Header_Len', 'Bwd_Header_Len', 'Fwd_Pkts/s',
       'Bwd_Pkts/s', 'Pkt_Len_Min', 'Pkt_Len_Max', 'Pkt_Len_Mean',
       'Pkt_Len_Std', 'Pkt_Len_Var', 'FIN_Flag_Cnt', 'SYN_Flag_Cnt',
       'RST_Flag_Cnt', 'PSH_Flag_Cn

In [58]:
columns_to_drop = [
    'Flow_ID',   
    'Src_IP',     
    'Dst_IP',     
    'Timestamp',  
    'Bwd_PSH_Flags',
    'Fwd_URG_Flags',
    'Bwd_URG_Flags',
    'CWE_Flag_Count',
    'ECE_Flag_Cnt',
    'Fwd_Byts/b_Avg',
    'Fwd_Pkts/b_Avg',
    'Fwd_Blk_Rate_Avg',
    'Bwd_Byts/b_Avg',
    'Bwd_Pkts/b_Avg',
    'Bwd_Blk_Rate_Avg',   
    'Cat'
]

df_cls = df.drop(columns=columns_to_drop, axis=1)

# Loại bỏ các mẫu có Label = 'Normal'
df_cls = df_cls[df_cls['Label'] != 'Normal']

print("Các cột đã được loại bỏ thành công. Số cột còn lại:", df_cls.shape[1])
print("\nDanh sách các cột sau khi loại bỏ:")
print(df_cls.columns)
print("Số mẫu còn lại sau khi loại bỏ Normal:", df_cls.shape[0])

Các cột đã được loại bỏ thành công. Số cột còn lại: 70

Danh sách các cột sau khi loại bỏ:
Index(['Src_Port', 'Dst_Port', 'Protocol', 'Flow_Duration', 'Tot_Fwd_Pkts',
       'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts', 'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Max',
       'Fwd_Pkt_Len_Min', 'Fwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std',
       'Bwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Std', 'Flow_Byts/s', 'Flow_Pkts/s', 'Flow_IAT_Mean',
       'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Tot',
       'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Fwd_IAT_Min',
       'Bwd_IAT_Tot', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Bwd_IAT_Max',
       'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Fwd_Header_Len', 'Bwd_Header_Len',
       'Fwd_Pkts/s', 'Bwd_Pkts/s', 'Pkt_Len_Min', 'Pkt_Len_Max',
       'Pkt_Len_Mean', 'Pkt_Len_Std', 'Pkt_Len_Var', 'FIN_Flag_Cnt',
       'SYN_Flag_Cnt', 'RST_Flag_Cnt', 'PSH_Flag_Cnt', 'ACK_Flag_Cnt',
       'URG_Flag_Cnt', 'Down/Up_Ratio', 'Pkt_Size_Avg', 'Fwd_Seg

In [59]:
# Kiểm tra số lượng loại nhãn trong Label
print("Các giá trị nhãn Label và số lượng:")
print(df['Label'].value_counts())

Các giá trị nhãn Label và số lượng:
Label
Anomaly    585710
Normal      40073
Name: count, dtype: int64


In [60]:
# Train model to classify Sub_Cat
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Loại bỏ các mẫu không có Sub_Cat hoặc Sub_Cat bị thiếu
cls_data = df_cls.dropna(subset=['Sub_Cat'])

# Encode target
le = LabelEncoder()
cls_data['subcat_encoded'] = le.fit_transform(cls_data['Sub_Cat'].astype(str))

# Chọn feature, loại bỏ cột target và các cột không cần thiết
X = cls_data.drop(columns=['Sub_Cat', 'Label'])
# Encode toàn bộ feature về số
for col in X.columns:
    X[col] = LabelEncoder().fit_transform(X[col].astype(str))

y = cls_data['subcat_encoded']

# Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.9979341312253505
Confusion matrix:
 [[11864    10     4     0     0     0     0     0]
 [    1  7066     8     0     0     0     0     0]
 [    0     0 10959    66     0     0     0     0]
 [    0     0    82 11082     0     0     0     0]
 [    0     1     0    31 24204     0     0     0]
 [    0     0     0     0     0 36711     0     0]
 [    0     0     0     0     7    11  4408    12]
 [    0     0     0     0     0     2     7 10606]]
                       precision    recall  f1-score   support

      DoS-Synflooding       1.00      1.00      1.00     11878
    MITM ARP Spoofing       1.00      1.00      1.00      7075
    Mirai-Ackflooding       0.99      0.99      0.99     11025
  Mirai-HTTP Flooding       0.99      0.99      0.99     11164
Mirai-Hostbruteforceg       1.00      1.00      1.00     24236
   Mirai-UDP Flooding       1.00      1.00      1.00     36711
        Scan Hostport       1.00      0.99      1.00      4438
         Scan Port OS       1.00      